<a href="https://colab.research.google.com/github/sajjkavinda/rsna-classification/blob/dataset-subesetting/rsna_subsetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rsna_2022_cervical_spine_fracture_detection_path = kagglehub.competition_download('rsna-2022-cervical-spine-fracture-detection')
samuelcortinhas_rsna_2022_spine_fracture_detection_metadata_path = kagglehub.dataset_download('samuelcortinhas/rsna-2022-spine-fracture-detection-metadata')

print('Data source import complete.')


In [ ]:
import os

base_dir = "/kaggle/input/rsna-2022-cervical-spine-fracture-detection/"
print(os.listdir(base_dir))


In [ ]:
import pandas as pd

labels = pd.read_csv("/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv")
print(labels.columns)

In [ ]:
import pandas as pd
import shutil
import os

# labels
labels = pd.read_csv(base_dir + "train.csv")

# Randomly picking 10
fractured = labels[labels["patient_overall"]==1]["StudyInstanceUID"].sample(10, random_state=42)
normal = labels[labels["patient_overall"]==0]["StudyInstanceUID"].sample(10, random_state=42)

subset = list(fractured) + list(normal)

# Output
os.makedirs("/kaggle/working/rsna_subset", exist_ok=True)

# Copy to a new folder
for pid in subset:
    src = os.path.join(base_dir, "train_images", pid)
    dst = os.path.join("/kaggle/working/rsna_subset", pid)
    shutil.copytree(src, dst)


In [ ]:
import pydicom
pydicom.config.image_handlers = ['pylibjpeg']

print("Active handlers:", pydicom.config.image_handlers)

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
import os

png_root = "/kaggle/working/rsna_subset_png"

def normalize_image(img):
    # Contrast stretching to 0–255
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    return img.astype(np.uint8)

for pid in tqdm(os.listdir(png_root), desc="Normalizing PNGs"):
    folder = os.path.join(png_root, pid)
    for f in os.listdir(folder):
        if f.endswith(".png"):
            path = os.path.join(folder, f)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            norm = normalize_image(img)
            cv2.imwrite(path, norm)
